In [ ]:
bva = 0 
for x in range(0, 50):
    minsamp = 0 + (x*500)
    print("next", x)
    print()
    for t in range(0, 50):
        m = 0
        
        if t < 10: 
            m = 0+t
            print
        elif t <= 20:
            m = ((t%10)*10)
        elif t <= 30:
            m = ((t%10)*100)
        elif t <= 40:
            m = ((t%10)*1000)
        elif t <= 50:
            m = ((t%10)*10000)
            
        epss =  .0001 + (.0001 * m)
        db = DBSCAN(eps = epss, min_samples = minsamp).fit(points)
       

        lab = db.labels_
        colors = []

        for i in range(0, len(lab)):
            if lab[i] == -1:
                colors.append('#000000')
            if lab[i] == 0:
                colors.append('#38FF00')
            if lab[i] == 1:
                colors.append('#F4FF00')
            if lab[i] == 2:
                colors.append('#FF0000')        
            if lab[i] > 2:
                colors.append('#FF00F8')        
        
        if (len(set(colors))) > bva: 
            print("found one" ,t,x, len(set(colors)))
            bva = len(set(colors)) 
        if (len(set(colors))) < bva: 
            print("stop" ,minsamp,x, len(set(colors)))
            bva = len(set(colors)) 

In [ ]:
# this is the final Code ie the only cell that matters 

import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, AffinityPropagation, MeanShift, estimate_bandwidth
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imread
from sklearn import cluster, datasets, mixture
import math
np.set_printoptions(threshold=10000)



# read in dataset
print("starting...")
ds = pd.read_csv('Crime_Data_Clean_Consolidated.csv')
first10 = ds.head(1200000)
(first10['Location']) 
loc = pd.Series(np.concatenate([str(x).strip().replace('(', '').replace(')', '').replace(' ', '').replace('-', '').split(',')for x in first10.Location]))






#-----------------------control panel Start---------------------------------------------------

extradata = True # turn this to false to bypass extra data
sizeDots = 5
maxNumClusters = 50
eps = .01

# data select 
toremove = []
testdata = first10.applymap(str)['CrimeCode']#change this stuff !!!!!
#testdata2 = first10.applymap(str)['UnemploymentRate']
for i in range(0, len(testdata)):
        if testdata[i] == '480':
            toremove.append(i)

    

#------------------------control panel end ---------------------------------------------------------    
    
# build the Xs and Ys
print("num of data",len(toremove))
loc1 = loc.values.tolist()
X = [[]]
Y = [[]]
size = len(loc1) 
for i in range(0,size): 
 
    if (i % 2) != 0:
        if loc1[i] != []:
            X.append(float(loc1[i]))
        else:
            print(i)
    else:
        if loc1[i] != []:
            Y.append(float(loc1[i]))
        else:
            print(i)  
X = np.asarray(X)
Y = np.asarray(Y)
X = np.nan_to_num(X)
Y = np.nan_to_num(Y)
X = X.tolist()
Y = Y.tolist()
X.remove([])
Y.remove([])
print("built X and Y...")
newX = []
newY = []
for i in toremove:
    newX.append(X[int(i)])
    newY.append(Y[int(i)])
print(len(X),len(Y))
print("appended toremove...")
# remove 0s      
while 0 in newX:
    newX.remove(0)
while 0 in newY:
    newY.remove(0)        
X = np.asarray(newX)
Y = np.asarray(newY)
temp = []
for i in range(0,len(X)):
    temp.append([X[i], Y[i]])
points = np.asarray(temp)




#DBSCAN 
print("starting dbscan...")
db = DBSCAN(eps= eps, min_samples = maxNumClusters, algorithm='ball_tree', 
            metric='euclidean').fit(points)

lab = db.labels_
print("number of clusters: ",len(set(lab)))
midsX = []
midsY = []


# extra data section 
if extradata == True:
    X = X.tolist()
    Y = Y.tolist()
    for i in range(0, len(set(lab))):
        totalX = 0
        totalY = 0 
        n = 0
        for x in range(0,len(lab)):
            if lab[x] == i:
                n = n+1
                totalX = totalX + X[x]
                totalY = totalY + Y[x]
        if n != 0:
            totalX = totalX / n
            totalY = totalY / n
            midsX.append(totalX)
            midsY.append(totalY)
            totdis = 0
            for x in range(0, len(X)):
                dis = math.sqrt((X[x]-totalX)**2 + (Y[x]-totalY)**2)
                totdis = totdis + dis
            totdis = totdis / n 
            weighted = totdis * ((len(X) - n) / len(X))
            print("for cluster:",i," average distance :", totdis, " weighted average distance :",weighted)

X = np.asarray(X)
Y = np.asarray(Y)
    
colors = []

for i in range(0, len(lab)):
    if lab[i] == -1:
        colors.append('#000000')
    if lab[i] == 0:
        colors.append('#38FF00') #light green
    if lab[i] == 1:
        colors.append('#F4FF00')#yellow
    if lab[i] == 2:
        colors.append('#FF0000')   #red     
    if lab[i] == 3:
        colors.append('#FF00F8')   #magenta      
    if lab[i] == 4:
        colors.append('#0FCF80') #darker green
    if lab[i] == 5:
        colors.append('#0427FC')#dark blue
    if lab[i] >= 6:
        colors.append('#8B04FC')  # purple       
        

print("number of clusters: ",len(set(lab)))
print("building plot")

plt.figure(figsize=(60,50))
img = imread("PIA03372.jpg")
plt.scatter(X,Y,s = sizeDots,c = colors, alpha = .5, zorder =1)
plt.imshow(img, zorder =  0,extent = [118.7058, 117.7024, 33.5368, 34.78033])#, extent = [118.7058, 117.7024, 33.5368, 34.78033])
plt.show()